In [73]:
import tensorflow as tf
import numpy as np
tf.set_random_seed(777)

In [109]:
tf.reset_default_graph()

In [110]:
sample = ' This is work in progress, and may have bugs. However, we call for your comments and pull requests. Check out our style guide line'
len(sample)

130

In [111]:
idx2char = list(set(sample))
idx2char

['d',
 'i',
 '.',
 'a',
 'm',
 'T',
 't',
 'p',
 ' ',
 'u',
 'q',
 's',
 'e',
 'n',
 'f',
 'w',
 ',',
 'o',
 'c',
 'l',
 'C',
 'k',
 'y',
 'r',
 'h',
 'H',
 'b',
 'v',
 'g']

In [112]:
char2idx = {c: i for i, c in enumerate(idx2char)}
char2idx

{' ': 8,
 ',': 16,
 '.': 2,
 'C': 20,
 'H': 25,
 'T': 5,
 'a': 3,
 'b': 26,
 'c': 18,
 'd': 0,
 'e': 12,
 'f': 14,
 'g': 28,
 'h': 24,
 'i': 1,
 'k': 21,
 'l': 19,
 'm': 4,
 'n': 13,
 'o': 17,
 'p': 7,
 'q': 10,
 'r': 23,
 's': 11,
 't': 6,
 'u': 9,
 'v': 27,
 'w': 15,
 'y': 22}

In [113]:
dic_size = len(char2idx)
hidden_size = len(char2idx)
num_classes = len(char2idx)
batch_size = 1
sequence_length = len(sample) - 1
learning_rate = 0.1

In [114]:
sample_idx = [char2idx[c] for c in sample]
sample_idx

[8,
 5,
 24,
 1,
 11,
 8,
 1,
 11,
 8,
 15,
 17,
 23,
 21,
 8,
 1,
 13,
 8,
 7,
 23,
 17,
 28,
 23,
 12,
 11,
 11,
 16,
 8,
 3,
 13,
 0,
 8,
 4,
 3,
 22,
 8,
 24,
 3,
 27,
 12,
 8,
 26,
 9,
 28,
 11,
 2,
 8,
 25,
 17,
 15,
 12,
 27,
 12,
 23,
 16,
 8,
 15,
 12,
 8,
 18,
 3,
 19,
 19,
 8,
 14,
 17,
 23,
 8,
 22,
 17,
 9,
 23,
 8,
 18,
 17,
 4,
 4,
 12,
 13,
 6,
 11,
 8,
 3,
 13,
 0,
 8,
 7,
 9,
 19,
 19,
 8,
 23,
 12,
 10,
 9,
 12,
 11,
 6,
 11,
 2,
 8,
 20,
 24,
 12,
 18,
 21,
 8,
 17,
 9,
 6,
 8,
 17,
 9,
 23,
 8,
 11,
 6,
 22,
 19,
 12,
 8,
 28,
 9,
 1,
 0,
 12,
 8,
 19,
 1,
 13,
 12]

In [115]:
x_data = [sample_idx[:-1]]
y_data = [sample_idx[1:]]
x_data

[[8,
  5,
  24,
  1,
  11,
  8,
  1,
  11,
  8,
  15,
  17,
  23,
  21,
  8,
  1,
  13,
  8,
  7,
  23,
  17,
  28,
  23,
  12,
  11,
  11,
  16,
  8,
  3,
  13,
  0,
  8,
  4,
  3,
  22,
  8,
  24,
  3,
  27,
  12,
  8,
  26,
  9,
  28,
  11,
  2,
  8,
  25,
  17,
  15,
  12,
  27,
  12,
  23,
  16,
  8,
  15,
  12,
  8,
  18,
  3,
  19,
  19,
  8,
  14,
  17,
  23,
  8,
  22,
  17,
  9,
  23,
  8,
  18,
  17,
  4,
  4,
  12,
  13,
  6,
  11,
  8,
  3,
  13,
  0,
  8,
  7,
  9,
  19,
  19,
  8,
  23,
  12,
  10,
  9,
  12,
  11,
  6,
  11,
  2,
  8,
  20,
  24,
  12,
  18,
  21,
  8,
  17,
  9,
  6,
  8,
  17,
  9,
  23,
  8,
  11,
  6,
  22,
  19,
  12,
  8,
  28,
  9,
  1,
  0,
  12,
  8,
  19,
  1,
  13]]

In [116]:
X = tf.placeholder(tf.int32,[None,sequence_length])
Y = tf.placeholder(tf.int32,[None,sequence_length])

In [117]:
x_one_hot = tf.one_hot(X,num_classes)

In [118]:
cell = tf.nn.rnn_cell.BasicLSTMCell(num_units=hidden_size,state_is_tuple=True)
initial_state = cell.zero_state(batch_size,tf.float32)

In [119]:
outputs, states = tf.nn.dynamic_rnn(cell,x_one_hot,initial_state=initial_state
                                   , dtype=tf.float32)

In [120]:
X_for_fc = tf.reshape(outputs,[-1,hidden_size])
outputs = tf.contrib.layers.fully_connected(X_for_fc,num_classes,activation_fn=None)

In [121]:
outputs = tf.reshape(outputs,[batch_size,sequence_length,num_classes])
weights = tf.ones([batch_size,sequence_length])

In [122]:
sequence_loss = tf.contrib.seq2seq.sequence_loss(logits=outputs,
                                                targets=Y,weights=weights)
loss = tf.reduce_mean(sequence_loss)
train = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)

In [123]:
prediction = tf.argmax(outputs,axis=2)

In [125]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for i in range(50):
        l,_ = sess.run([loss,train],feed_dict={X:x_data,Y:y_data})
        result = sess.run(prediction,feed_dict={X:x_data})
        result_str = [idx2char[c] for c in np.squeeze(result)]
        print(i,"loss :",l,"prediction:",''.join(result_str))

0 loss : 3.3785503 prediction:                    rr                                                                                                            
1 loss : 3.2060938 prediction:                                                                                                                                  
2 loss : 3.0413232 prediction: s    ssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssss
3 loss : 3.0544946 prediction: oii                                                            o  oo   o                 o   o                   o               
4 loss : 2.8949838 prediction: eee                                  e            e                        a                e                                    
5 loss : 2.8170676 prediction: ehe                                e e            e                  e    ea                u       e                            
6 loss : 2.684759 prediction: uhe 